## Ejercicio 4: Convertir datos financieros estructurados en un informe narrativo

Partiendo de un dataframe con datos financieros, usaremos la API de OpenAI para generar un reporte en lenguaje natural

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

print(type(OPENAI_API_KEY),OPENAI_API_KEY)
print(type(API_VERSION),API_VERSION)
print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [ ]:
import pandas as pd
import numpy as np
from openai import AzureOpenAI
import json

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

In [ ]:
np.random.seed(42)
num_filas = 100  # Número de registros

# Generamos fechas aleatorias entre el 01/01/2023 y el 31/12/2024
fechas = pd.to_datetime(
    np.random.choice(pd.date_range("2023-01-01", "2024-12-31"), size=num_filas)
)

# Generamos datos simulados
ingresos = np.random.randint(100000, 500000, size=num_filas)
gastos = np.random.randint(80000, 400000, size=num_filas)
balance = ingresos - gastos
numero_clientes = np.random.randint(1000, 10000, size=num_filas)
costos_operativos = np.random.randint(50000, 150000, size=num_filas)

# Creamos el DataFrame
df_financiero = pd.DataFrame({
    "Fecha": fechas,
    "Ingresos": ingresos,
    "Gastos": gastos,
    "Balance": balance,
    "Numero_de_clientes": numero_clientes,
    "Costos_operativos": costos_operativos
})

df_financiero = df_financiero.sort_values("Fecha").reset_index(drop=True)

# Guardamos el DataFrame en un archivo CSV
df_financiero.to_csv("financial_data.csv", index=False, encoding="utf-8")
print("Archivo 'financial_data.csv' generado exitosamente.\n")

##### Leemos el csv como dataframe de pandas, y lo convertimos en json

In [ ]:
df_report = pd.read_csv("financial_data.csv", parse_dates=["Fecha"])
display(df_report.head())

datos_json = df_report.to_json(orient="records", date_format="iso", force_ascii=False)
print(datos_json)

##### Definimos el prompt

In [ ]:
prompt_informe = (
    "Eres un analista financiero con amplia experiencia en la elaboración de informes detallados. "
    "A partir de los siguientes datos financieros, debes generar un informe narrativo extenso que contenga las siguientes secciones:\n\n"
    "1. **Resumen Ejecutivo:** Resume la situación general y las tendencias globales observadas en los datos. "
    "Destaca los periodos de mayor rendimiento y las áreas de oportunidad.\n\n"
    "2. **Análisis de Ingresos y Gastos:** Describe en detalle cómo varían los ingresos y gastos a lo largo del tiempo. "
    "Identifica periodos con altas discrepancias y analiza posibles causas de dichas variaciones.\n\n"
    "3. **Evaluación del Balance:** Explica la evolución del balance resultante de la diferencia entre ingresos y gastos. "
    "Relaciona estos cambios con el comportamiento de los otros indicadores.\n\n"
    "4. **Impacto en la Base de Clientes:** Analiza cómo el número de clientes podría influir en los resultados financieros. "
    "Comenta sobre la posible relación entre el crecimiento o decrecimiento de clientes y el rendimiento financiero.\n\n"
    "5. **Costos Operativos y Eficiencia:** Examina los costos operativos y su impacto en la eficiencia global del banco. "
    "Incluye observaciones sobre la optimización de procesos y la reducción de gastos innecesarios.\n\n"
    "6. **Conclusiones y Recomendaciones:** Concluye el informe con un resumen de los hallazgos y proporciona recomendaciones "
    "estratégicas basadas en el análisis realizado.\n\n"
    "Utiliza un lenguaje técnico pero claro, adecuado para un público de profesionales financieros. "
    "Asegúrate de que el informe sea coherente, estructurado y que cada sección se conecte lógicamente con la siguiente.\n\n"
    "A continuación, se presentan los datos financieros en formato JSON:\n"
    f"{datos_json}\n\n"
    "Informe narrativo:"
)

##### Creamos el informe

In [ ]:
from IPython.display import Markdown

client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_informe}],
        temperature=0.5,
        max_tokens=2000,
    )

informe_narrativo = response.choices[0].message.content.strip()
print("Informe narrativo generado:\n")
display(Markdown(informe_narrativo))
#si no funciona, descomentar esta línea:
#print(informe_narrativo)

### Podemos ver también el coste que ha tenido la llamada

In [ ]:
response

In [ ]:
def calcular_coste_uso_api(response, modelo: str) -> dict:
    """
    Calcula el costo de uso de la API de Azure OpenAI basado en los tokens usados,
    separando el coste del input (prompt) y del output (completion).

    Para el modelo "gpt-4o":
      - Input: 2.50 $ por millón de tokens.
      - Output: 10.00 $ por millón de tokens.
    (Referencia: https://openai.com/api/pricing/)

    Args:
        response: Objeto de respuesta de la API que contiene el atributo 'usage'.
        modelo (str): Nombre del modelo utilizado (ej: "gpt-4o-mini").

    Returns:
        dict: Diccionario con los costos calculados para entrada, salida y el total.
    """
    #Definimos los precios para el modelo gpt-4o
    if "gpt-4o" in modelo:
        coste_prompt_millon = 2.50 # $ por 1,000,000 tokens de entrada
        coste_completion_millon = 10.00  # $ por 1,000,000 tokens de salida
    else:
        print("Modelo no reconocido.")
        return {}

    #Extraemos los tokens usados del atributo 'usage' del objeto de respuesta (se podría devolver también en el return si se desea)
    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens

    #Calculamos el coste para cada parte (teniendo en cuenta que el precio es por millón de tokens)
    coste_prompt = (prompt_tokens / 1000000) * coste_prompt_millon
    coste_completion = (completion_tokens / 1000000) * coste_completion_millon
    coste_total = coste_prompt + coste_completion

    return {
        "coste_prompt": coste_prompt,
        "coste_completion": coste_completion,
        "coste_total": coste_total
    }

# Ejemplo de uso con un objeto de respuesta simulado similar al que describes:
class Usage:
    def __init__(self, prompt_tokens, completion_tokens):
        self.prompt_tokens = prompt_tokens
        self.completion_tokens = completion_tokens

class ChatCompletion:
    def __init__(self, usage):
        self.usage = usage

In [ ]:
model = "gpt-4o"
cost = calcular_coste_uso_api(response, model)

print(f"Coste de tokens de entrada (prompt): ${cost['coste_prompt']:.6f}")
print(f"Coste de tokens de salida (completion): ${cost['coste_completion']:.6f}")
print(f"Coste total: ${cost['coste_total']:.6f}")

##### - ¿Cómo afecta la estructura del prompt al nivel de detalle y coherencia del informe narrativo generado?
##### - ¿Qué mejoras o ajustes sugerirías para obtener un informe aún más preciso o enfocado en ciertos aspectos?